# Estimating Spectra

trying different estimators for spectra

In [ ]:
import os
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import mtspec
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
matplotlib.rc_file('../rc_file_paper')
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
sys.path.append("..")
from paths import path_samoc, path_results
import statsmodels.tsa as tsa
from statsmodels.tsa.arima_process import ArmaProcess

## Why is the periodogram is a bad estimator?
1. not robust: more data points do not lead to a decrease in the estimate variance
1. spectral leakage: intensitiesare contaminated with variance from far aways frequencies. Can be solved with tapers.

I will use three time series with known spectra to illustrate the problems and compare different spectral estimators.
1. `A` white noise time series: theoretical power spectrum $\Gamma(\omega) = \mathrm{var}(A)$
1. `B` AR(2) process ($\alpha_1=0.3, \alpha_2=0.3$)
1. `C` AR(2) process ($\alpha_1=0.9, \alpha_2=-0.8$)

The theoretical power spectrum of an AR(2) process is
$$\Gamma(\omega) = \frac{\sigma_Z^2}{1+\alpha_1^2+\alpha_2^2-2 g(\omega)}$$
with  $ g ( \omega ) = \alpha_1 (1 - \alpha_2) \cos(2 \pi \omega) + \alpha_2 \cos(4 \pi \omega)$ (p. 224 von Storch and Zwiers)


Ghil et al. (2000): 
> For instrumental climate records, with a typical length of a few hundred points, the choice p ⫽ 2 and K ⫽ 3 offers a good compromise between the required frequency resolution for resolving distinct cli- mate signals (e.g., ENSO and decadal-scale variability) and the benefit of multiple spectral degrees of freedom, i.e., of reduced variance [e.g., Mann and Park, 1993].

In [ ]:
def periodogram(ts, d=1.):
    """ naive absolute squared Fourier components 
    
    input:
    ts .. time series
    d  .. sampling period
    """
    freq = np.fft.rfftfreq(len(ts))
    Pxx = 2*np.abs(np.fft.rfft(ts))**2/len(ts)
    return freq, Pxx

def Welch(ts, d=1, window='hann'):
    """ Welch spectrum """
    return sp.signal.welch(ts, window=window)

def mtspectrum(ts, d=1., tb=4, nt=4):
    """ multi-taper spectrum 
    
    input:
    ts .. time series
    d  .. sampling period
    tb .. time bounds (bandwidth)
    nt .. number of tapers
    """
    spec, freq, jackknife, _, _ = mtspec.mtspec(
                data=ts, delta=d, time_bandwidth=tb,
                number_of_tapers=nt, statistics=True)
    return freq, spec

In [ ]:
def AR2_process(a1, a2, N):
    """ generates AR(2) time series of length N """
    return ArmaProcess(np.array([1, -a1, -a2]), np.array([1])).generate_sample(nsample=N)

def AR2_spectrum(sz, a1, a2, omega):
    """ theoretical AR(2) spectrum 
    
    input:
    sz     .. standard deviation
    a1, a2 .. AR(2) coefficients
    omega  .. frequency at which to evaluate expression
    """
    # multiplied here by an extra factor of 2, so that we can integrate over [0,0.5] and not over [-0.5.0.5]
    return 2*sz**2/(1+a1**2+a2**2-2*(a1*(1-a2)*np.cos(2*np.pi*omega)+a2*np.cos(4*np.pi*omega)))

N = 240
A = np.random.rand((N))-.5
A /= np.std(A)
b = 0.3
B_ = AR2_process(b, b, N)
B_ /= np.std(B_)
B = B_ + 10*np.sin(2*np.pi*.3162*np.arange((N))+.5763)
# B /= np.std(B)
c1, c2 = .9, -.8
C = AR2_process(c1, c2, N)
C /= np.std(C)

In [ ]:
f, ax = plt.subplots(5,3,figsize=(12,12), constrained_layout=True)
for i, ts in enumerate([A,B,C]):
    ax[0,i].axhline(0, c='grey', lw=.5)
    ax[0,i].plot(np.arange(N), ts)
    
    for j, ts_ in enumerate([ts[:120], ts]):
        ts_ -= np.mean(ts_)
        ax[j+1,i].plot([0,0],[0,1], c='w', label=f'var(x)={np.var(ts_):7.3f}')
        for k in [0,2]:
            freq = np.linspace(0,.5,len(ts_))
            if i==0:  
                ax[j+1+k,i].axhline(2*np.var(ts_), c='r', label=f'theor. {np.var(ts_):7.3f}')
                # multiplied here by an extra factor of 2, so that we can integrate over [0,0.5] and not over [-0.5.0.5]
            if i==1:
                spec = AR2_spectrum(sz=1, a1=b , a2=b , omega=freq)
                ax[j+1+k,i].plot(freq, spec, c='r', label=f'theor. {np.sum((freq[1]-freq[0])*spec):7.3f}')
                ax[j+1+k,i].axvline(.3162, c='r')
            
            if i==2:
                spec = AR2_spectrum(sz=1 , a1=c1, a2=c2, omega=freq)
                ax[j+1+k,i].plot(freq, 10*np.log10(spec), c='r', label=f'theor. {np.sum((freq[1]-freq[0])*spec):7.3f}')
                ax[j+1+k,i].axhline(0, c='r')
            
        freq, Pxx = periodogram(ts_)
        for k in [0,2]:
            if i<2:  ax[j+1+k,i].bar(freq[1:], Pxx[1:], width=.001, label=f'periodogram {np.sum((freq[1]-freq[0])*Pxx):7.3f}')
            else  :  ax[j+1+k,i].bar(freq[1:], 10*np.log10(Pxx[1:]), width=.003, label=f'periodogram {np.sum((freq[1]-freq[0])*Pxx):7.3f}')
        
        
        for window in ['hann', 'boxcar', 'bartlett']:
            f, Pxx = sp.signal.welch(ts, window=window, nperseg=60)#, average='median')
            if i<2:  ax[j+1,i].scatter(f[1:], Pxx[1:], label=f'Welch {window} {np.sum((f[1]-f[0])*Pxx):7.3f}')
            else  :  ax[j+1,i].scatter(f[1:], 10*np.log10(Pxx[1:]), label=f'Welch {window} {np.sum((f[1]-f[0])*Pxx):7.3f}')
        
        
        
        freq, spec = mtspectrum(ts_)
        if i<2:  ax[j+3,i].scatter(freq[1:], spec[1:], label=f'MT spec {np.sum((freq[1]-freq[0])*spec):7.3f}')
        else  :  ax[j+3,i].scatter(freq[1:], 10*np.log10(spec[1:]), label=f'MT spec {np.sum((freq[1]-freq[0])*spec):7.3f}')
        
        for k in [0,2]:
            if i==0: ax[j+1+k,i].set_ylim((0,8))
            if i==1: ax[j+1+k,i].set_ylim((0,20))
#         if i==2: ax[j+1,i].set_yscale('log', basey=10)
        ax[j+1,i].legend(fontsize=8)
        ax[j+3,i].legend(fontsize=8)
        
for i in range(3):
    ax[0,i].set_title(['white noise', r'AR(2) (0.3,0.3) + $10 \sin(2 \pi 0.3162 t + 0.5763)$','AR(2) (0.9,-0.8)'][i])
    ax[i,0].set_ylabel([r'time series $x(t)$\namplitude', 'n=120\npower spectral density','n=240\npower spectral density'][i])

## index time series

In [ ]:
# das = []
# for idx in ['AMO', 'TPI', 'SOM']:
#     for run in ['had', 'ctrl', 'lpd']:
#         if run=='ctrl':   ts, Nt = '_51_301', 250*12
#         elif run=='lpd':  ts, Nt = '_154_404', 250*12
#         elif run=='had':  ts, Nt = '', 149*12
#         fn = f'{path_samoc}/SST/{idx}_ds_dt_raw_{run}{ts}.nc'
#         print(os.path.exists(fn))
#         da = xr.open_dataarray(fn, decode_times=False).assign_coords(time=np.arange(Nt))
#         da.name = f'{idx}_{run}'
#         das.append(da)

In [ ]:
ds = xr.open_dataset(f'{path_results}/all_timeseries.nc')

In [ ]:
fig, ax = plt.subplots(4,3,figsize=(12,15),constrained_layout=True, sharey='row')
for i, idx in enumerate(['AMO', 'TPI', 'SOM']):
    for j, run in enumerate(['had', 'ctrl', 'lpd']):
        c = f'C{i}'
        key = f'{idx}_{run}'
        x = ds[key].dropna(dim='time')
        f = np.linspace(0,0.5,len(x))
        std_ = ds[key].std().values
        AM = ARMA(endog=x.values, order=(1,0)).fit()
        ac, std = AM.arparams[0], np.sqrt(AM.sigma2)
        ax[0,j].plot(ds.time, ds[key]-i, label=fr'{idx} $\sigma_x=${std_:.2f} $\sigma_\epsilon=${std:.2f}', c=c)
        ax[0,j].axhline(-i, c=c, lw=.5)
        
        for k in range(3):  ax[k+1,j].plot(f[1:], AR2_spectrum(sz=std, a1=ac, a2=0, omega=f)[1:]*10**(-[3,1.5,2][k]*i), c=c, ls='-.')
        
        f, Pxx = periodogram(x)
        ax[1,j].plot(f[1:], Pxx[1:]*10**(-3*i), c=c, alpha=.5, label=fr'{idx}')#' $\rho_1$={ac:.3f}')
        mc = mc_ar1_spectrum(x.values, spectrum=periodogram)
        ax[1,j].plot(mc['freq'][1:], mc['median'][1:]*10**(-3*i), c=c, ls='-' , label='median')
        ax[1,j].plot(mc['freq'][1:], mc['5']     [1:]*10**(-3*i), c=c, ls='--', label='5%')
        ax[1,j].plot(mc['freq'][1:], mc['1']     [1:]*10**(-3*i), c=c, ls=':' , label='1%')
        ax[1,j].plot(mc['freq'][1:], mc['95']    [1:]*10**(-3*i), c=c, ls='--', label='95%')
        ax[1,j].plot(mc['freq'][1:], mc['99']    [1:]*10**(-3*i), c=c, ls=':' , label='99%')
        
        f, Pxx = Welch(x)
        ax[2,j].plot(f[1:], Pxx[1:]*10**(-1.5*i), c=c, alpha=.5, label=fr'{idx} $\rho_1$={ac:.3f}')
#         mc = mc_ar1_spectrum(x.values, spectrum=Welch)
#         ax[1,j].plot(mc['freq'][1:], mc['median'][1:]*10**(-1.5*i), c=c, alpha=.5, label='median')
#         ax[1,j].plot(mc['freq'][1:], mc['95'][1:]*10**(-1.5*i), c=c, alpha=.5, label='95%')
#         ax[1,j].plot(mc['freq'][1:], mc['99'][1:]*10**(-1.5*i), c=c, alpha=.5, label='99%')
        
        f, Pxx = mtspectrum(x)
        ax[3,j].plot(f[1:], Pxx[1:]*10**(-2*i), c=c, alpha=.5, label=fr'{idx} $\rho_1$={ac:.3f}')
        mc = mc_ar1_spectrum(x.values, spectrum=mtspectrum)
        ax[3,j].plot(mc['freq'][1:], mc['median'][1:]*10**(-2*i), c=c, ls='-' , label='median')
        ax[3,j].plot(mc['freq'][1:], mc['5']     [1:]*10**(-2*i), c=c, ls='--', label='5%')
        ax[3,j].plot(mc['freq'][1:], mc['1']     [1:]*10**(-2*i), c=c, ls=':' , label='1%')
        ax[3,j].plot(mc['freq'][1:], mc['95']    [1:]*10**(-2*i), c=c, ls='--', label='95%')
        ax[3,j].plot(mc['freq'][1:], mc['99']    [1:]*10**(-2*i), c=c, ls=':' , label='99%')
        
for i in range(3):
    ax[0,i].set_title(['HIST', 'HIGH', 'LOW'][i])
    ax[0,i].set_xlabel(r'time [month]')
    ax[-1,i].set_xlabel(r'frequency [month$^{-1}$]')
    ax[1,i].set_ylim((1e-12, 1e2))
    ax[0,i].legend(fontsize=8)
    ax[1,i].legend(fontsize=7, ncol=3)
    for j in range(3):
        ax[j+1,i].set_xlim((3e-4, 6e-1))
        ax[j+1,i].set_xscale('log', basex=10)
        ax[j+1,i].set_yscale('log', basey=10)

for i in range(4):  ax[i,0].set_ylabel(['time series', 'periodogram', 'Welch', 'Multi-taper'][i])
plt.savefig(f'{path_results}/spectra/indices, spectra')

## Fitting AR(1) spectra: theoretical vs. MC spectral estimates

In [ ]:
from statsmodels.tsa.arima_model import ARMA

def mc_ar1_ARMA(phi, std, n, N=1000):
    """ Monte-Carlo AR(1) processes
    
    input:
    phi .. (estimated) lag-1 autocorrelation
    std .. (estimated) standard deviation of noise
    n   .. length of original time series
    N   .. number of MC simulations 
    """
    AR_object = ArmaProcess(np.array([1, -phi]), np.array([1]), nobs=n)
    mc = AR_object.generate_sample(nsample=(N,n), scale=std, axis=1, burnin=1000)
    return mc

def mc_ar1_spectrum(x, spectrum, N=1000, filter_type=None, filter_cutoff=None):
    """ calculates the Monte-Carlo spectrum and the 95% confidence interval
    
    input:
    x             .. time series
    spectrum      .. spectral density estimation function
    N             .. number of MC simulations
    filter_type   ..
    filter_cutoff ..
    
    output:
    mc_spectrum   .. 
    """
    AM = ARMA(endog=x, order=(1,0)).fit()
    phi, std = AM.arparams[0], np.sqrt(AM.sigma2)
    mc = mc_ar1_ARMA(phi=phi, std=std, n=len(x), N=N)
    mc_spectra = np.zeros((N, int(len(mc[0,:])/2)+1))
    for i in range(N):  freq, mc_spectra[i,:] = spectrum(mc[i,:])
    mc_spectrum = {}
    mc_spectrum['median'] = np.median(mc_spectra, axis=0)
    mc_spectrum['freq'] = freq
    for p in [1,2.5,5,95,97.5,99]:
        mc_spectrum[str(p)] = np.percentile(mc_spectra, p, axis=0)
    return mc_spectrum

## testing AR(1) spectral density estimates: theoretical vs. Monte-Carlo

In [ ]:
# long AR(1) processes and their MC spectra
f, ax = plt.subplots(3,1, figsize=(8,12))
for i, std in enumerate([.1, .5, .9]):
    n, phi = 10000, 0.5
    x = ArmaProcess(np.array([1, -phi]), np.array([1])).generate_sample(nsample=n, scale=std, burnin=1000)

    f, Pxx = mtspectrum(x)
    ax[i].plot(f, Pxx, alpha=.5)
    Pxx_th = AR2_spectrum(sz=std, a1=ac, a2=0, omega=f)
    ax[i].plot(f[1:], Pxx_th[1:], c='C3')

    mc_ = mc_ar1_spectrum(std=std, phi=phi, n=n, spectrum=mtspectrum)
    ax[i].plot(mc_[1,1:], mc_[0,1:], c='C1')
    ax[i].plot(mc_[1,1:], mc_[2,1:], c='C1')
    ax[i].plot(mc_[1,1:], mc_[3,1:], c='C1')

    AM = ARMA(endog=x, order=(1,0)).fit()
    ac, std = AM.arparams[0], np.sqrt(AM.sigma2)
    mc = mc_ar1_spectrum(std=std, phi=ac, n=len(x), spectrum=mtspectrum)
    ax[i].plot(mc[1,1:], mc[0,1:], c='C2', ls=':' , alpha=.8)
    ax[i].plot(mc[1,1:], mc[2,1:], c='C2', ls='--', alpha=.8)
    ax[i].plot(mc[1,1:], mc[3,1:], c='C2', ls='--', alpha=.8)
    ax[i].plot(mc[1,1:], mc[4,1:], c='C2', ls='--', alpha=.8)
    ax[i].plot(mc[1,1:], mc[5,1:], c='C2', ls='--', alpha=.8)

    ax[i].plot(f[1:], AR2_spectrum(sz=std, a1=ac, a2=0, omega=f)[1:], c='C3')

    ax[i].set_yscale('log', basey=10)

    print(f'{np.var(x):7.3f}, {np.sum((f[1]-f[0])*Pxx):7.3f}, {np.sum((f[1]-f[0])*Pxx_th):7.3f}, {np.sum((mc_[1,1]-mc_[1,0])*mc_[0,:]):7.3f}, {np.sum((mc[1,1]-mc[1,0])*mc[0,:]):7.3f}')

In [ ]:
# Yule Walker
n, phi, std = 1000, 0.5, .1
print(f'{phi:7.3f}, {std:7.3f}')
x = ArmaProcess(np.array([1, -phi]), np.array([1])).generate_sample(nsample=n, scale=std, burnin=1000)
print(f'{np.corrcoef(x[:-1], x[1:])[1,0]:7.3f}, {std:7.3f}')
# max likelihood estimation of parameters
AM = ARMA(endog=x, order=(1,0)).fit()
print(f'{AM.arparams[0]:7.3f}, {np.sqrt(AM.sigma2):7.3f}')


In [ ]:
fig, ax = plt.subplots(2,3,figsize=(12,6),constrained_layout=True, sharey='row')
for i, idx in enumerate(['AMO', 'TPI', 'SOM']):
    for j, run in enumerate(['had', 'ctrl', 'lpd']):
        c = f'C{i}'
        key = f'{idx}_{run}'
        AM = tsa.arima_model.ARMA(endog=ds[key].dropna(dim='time').values, order=(1,0)).fit()
        ac, std = AM.arparams[0], np.sqrt(AM.sigma2)
#         std = ds[key].std().values
#         ac = np.corrcoef(ds[key].dropna(dim='time').values[1:], ds[key].dropna(dim='time').shift(time=1).dropna(dim='time').values)[1,0]
        ax[0,j].plot(ds.time, ds[key]+i, label=fr'{idx} $\sigma=${std:.2f}', c=c)
        
        f, Pxx = mtspectrum(ds[key].dropna(dim='time').values)
        ax[1,j].plot(f[1:], Pxx[1:]*10**(2*i), c=c, alpha=.5, label=fr'{idx} $\rho_1$={ac:.3f}')
        ax[1,j].plot(f[1:], AR2_spectrum(sz=std, a1=ac, a2=0, omega=f)[1:]*10**(2*i), c=c)
        
        mc = mc_ar1_spectrum(std=std, phi=ac, n=len(ds[key].dropna(dim='time')), spectrum=mtspectrum)
        ax[1,j].plot(mc[1,1:], mc[0,1:]*10**(2*i), c=c, ls='-')
        ax[1,j].plot(mc[1,1:], mc[2,1:]*10**(2*i), c=c, ls='--')
        ax[1,j].plot(mc[1,1:], mc[3,1:]*10**(2*i), c=c, ls='--')
        ax[1,j].plot(mc[1,1:], mc[4,1:]*10**(2*i), c=c, ls=':')
        ax[1,j].plot(mc[1,1:], mc[5,1:]*10**(2*i), c=c, ls=':')
        

for i in range(3):
    ax[0,i].set_title(['HIST', 'HIGH', 'LOW'][i])
    ax[0,i].legend(fontsize=10)
    ax[1,i].legend(fontsize=10)
#     for j in range(3):
    ax[1,i].set_xlim((3e-4, 6e-1))
    ax[1,i].set_xscale('log', basex=10)
    ax[1,i].set_yscale('log', basey=10)

for i in range(2):  ax[i,0].set_ylabel(['time series', 'Multi-taper'][i])

In [ ]:
ac = .9
print(f'X_t       {0/(1-ac):6.3f}      {1/(1-ac**2):7.4f}')
for n in [10,100,1000,10000,100000,1000000]:
    a = ArmaProcess(np.array([1, -ac]), np.array([1])).generate_sample(nsample=n, burnin=1000)
    
    print(f'{n:7d}:  {np.mean(a):6.3f}      {np.var(a):7.4f}')
    

## testing $\int \hat{S}(f) \, \mathrm{d}f = \mathrm{var}(x(t))$

In [ ]:
def int_Sf(f, Pxx):
    df = f[1]-f[0]
    return np.sum(df*Pxx)

print('\033[1midx run   var.  perio Welch MTM   theor  MCper MCmtm    \033[0m')
for i, idx in enumerate(['AMO', 'TPI', 'SOM']):
    for j, run in enumerate(['had', 'ctrl', 'lpd']):
        c = f'C{i}'
        key = f'{idx}_{run}'
#         ac = np.corrcoef(ds[key].dropna(dim='time').values[1:], ds[key].dropna(dim='time').shift(time=1).dropna(dim='time').values)[1,0]
        x = ds[key].dropna(dim='time')
        var = ds[key].var().values
        AM = tsa.arima_model.ARMA(endog=ds[key].dropna(dim='time').values, order=(1,0)).fit()
        ac, std = AM.arparams[0], np.sqrt(AM.sigma2)
        f = np.linspace(0,.5,len(x))
        p = int_Sf(*periodogram(x))
        w = int_Sf(*Welch(x))
        m = int_Sf(*mtspectrum(x))
        t = int_Sf(f, AR2_spectrum(sz=std, a1=ac, a2=0, omega=f))
        c = int_Sf(f, mc_ar1_spectrum(x.values, spectrum=periodogram)['median'])
        d = int_Sf(f, mc_ar1_spectrum(x.values, spectrum=mtspectrum)['median'])
        print(f'{idx} {run:4} {var:.3f}  {p:.3f} {w:.3f} {m:.3f} {t:.3f}  {c:.3f} {d:.3f}    {var/c:7.3f} {var/d:7.3f}    {c/var:7.3f} {d/var:7.3f}')
    print()

- the equality holds for the periodogram, the Welch method (with some small error), and the multi-taper method
- theoretical and Monte-Carlo median spectra do not work correctly though!

In [ ]:
# theoretical vs. Monte-Carlo
n = 1000
f = np.linspace(0,.5,n)
df = f[1]-f[0]
fig, ax = plt.subplots(3,3, figsize=(8,5), sharey=True, sharex=True, constrained_layout=True)
for i, std in enumerate([0.1,.5,1]):
    for j, ac in enumerate([.1,.5,.9]):
        ax[i,j].plot([0,0],[0,1], c='w', label=fr'var$(X_t) = {std**2/(1-ac**2):.4f}$')

        spec = AR2_spectrum(sz=std, a1=ac, a2=0, omega=f)
        ax[i,j].plot(f, spec, label=fr'theor. $\int S = {np.sum(df*spec):.4f}$')
        
        mc_spec = mc_ar1_spectrum(phi=ac, std=std, n=n, spectrum=periodogram)
        ax[i,j].plot(mc_spec[1,:], mc_spec[0,:], c='C1', label=fr'median  $\int S = {np.sum(df*mc_spec[0,:]):.4f}$')
        ax[i,j].plot(mc_spec[1,:], mc_spec[2,:], c='C1', ls='--')  # 5%
        ax[i,j].plot(mc_spec[1,:], mc_spec[3,:], c='C1', ls='--')  # 95%
        
        
        ax[i,j].set_xscale('log', basex=10)
        ax[i,j].set_yscale('log', basey=10)
        ax[i,j].set_ylim((1e-4, 1e3))
        
        if i==0:  ax[i,j].set_title(fr'$\rho_1=${ac}', fontsize=16)
        if j==0:  ax[i,j].set_ylabel(fr'$\sigma=${std}', fontsize=16)
        ax[i,j].legend(fontsize=8)

In [ ]:
A = xr.DataArray(data=np.sin(np.arange(1000)) + .4*np.random.rand((1000)), dims='time', coords={'time':np.arange(1000)})

In [ ]:
plt.plot(A)

In [ ]:
from bb_analysis_timeseries import AnalyzeTimeSeries as ATS

In [ ]:
(spec, freq, jackknife) = ATS(A).spectrum()

In [ ]:
print(f'{freq[1]:e}')

In [ ]:
spec

In [ ]:
plt.scatter(freq, spec)
plt.axvline(1/2/np.pi)
# plt.loglog()

In [ ]:
from mtspec import mtspec
from mtspec.util import _load_mtdata

data = _load_mtdata('v22_174_series.dat.gz')

# Calculate the spectral estimation.
spec, freq, jackknife, _, _ = mtspec(
    data=data, delta=4930.0, time_bandwidth=3.5,
    number_of_tapers=5, nfft=312, statistics=True)

fig = plt.figure()
ax1 = fig.add_subplot(2, 1, 1)
# Plot in thousands of years.
ax1.plot(np.arange(len(data)) * 4.930, data, color='black')
ax1.set_xlim(0, 800)
ax1.set_ylim(-1.0, 1.0)
ax1.set_xlabel("Time [1000 years]")
ax1.set_ylabel("Change in $\delta^{18}O$")

ax2 = fig.add_subplot(2, 1, 2)
ax2.set_yscale('log')
# Convert frequency to Ma.
freq *= 1E6
ax2.plot(freq, spec, color='black')
ax2.fill_between(freq, jackknife[:, 0], jackknife[:, 1],
                 color="red", alpha=0.3)
ax2.set_xlim(freq[0], freq[-1])
ax2.set_ylim(0.1E1, 1E5)
ax2.set_xlabel("Frequency [c Ma$^{-1}]$")
ax2.set_ylabel("Power Spectral Density ($\delta^{18}O/ca^{-1}$)")

plt.tight_layout()


In [ ]:
len(data)

In [ ]:
len(spec)

In [ ]:
from paths import path_samoc

In [ ]:
AMO_ctrl = xr.open_dataarray(f'{path_samoc}/SST/AMO_ctrl.nc', decode_times=False)
AMO_rcp  = xr.open_dataarray(f'{path_samoc}/SST/AMO_rcp.nc' , decode_times=False)
AMO_lpi  = xr.open_dataarray(f'{path_samoc}/SST/AMO_lpi.nc' , decode_times=False)
AMO_lpd  = xr.open_dataarray(f'{path_samoc}/SST/AMO_lpd.nc' , decode_times=False)
AMO_had  = xr.open_dataarray(f'{path_samoc}/SST/AMO_had.nc' , decode_times=False)
AMOs = [AMO_ctrl, AMO_rcp, AMO_lpi, AMO_lpd, AMO_had]

In [ ]:
spec, freq, jackknife, _, _ = mtspec(
    data=AMO_ctrl.values, delta=1., time_bandwidth=2,
    number_of_tapers=8, statistics=True)
print(spec)
print(freq)

In [ ]:
# Calculate the spectral estimation.
spec, freq, jackknife, _, _ = mtspec(
    data=AMO_ctrl.values, delta=1., time_bandwidth=3.5,
    number_of_tapers=5, statistics=True)

fig = plt.figure()
ax1 = fig.add_subplot(2, 1, 1)
# Plot in thousands of years.
ax1.plot(np.arange(len(AMO_ctrl.values)), AMO_ctrl.values, color='black')

ax1.set_xlabel("Time [1000 years]")
ax1.set_ylabel("Change in $\delta^{18}O$")

ax2 = fig.add_subplot(2, 1, 2)
ax2.set_yscale('log')
# Convert frequency to Ma.
freq *= 1E6
ax2.plot(freq, spec, color='black')
ax2.fill_between(freq, jackknife[:, 0], jackknife[:, 1],
                 color="red", alpha=0.3)
ax2.set_xlim(freq[0], freq[-1])
ax2.set_xlabel("Frequency [c Ma$^{-1}]$")
ax2.set_ylabel("Power Spectral Density ($\delta^{18}O/ca^{-1}$)")

plt.tight_layout()


In [ ]:
# Calculate the spectral estimation.
fig, ax = plt.subplots(1, 1, figsize=(8,5))
ax.tick_params(labelsize=14)
for i in range(5):
    run = ['ctrl', 'rcp', 'lpd', 'lpi', 'had'][i]
    spec, freq, jackknife, _, _ = mtspec(
        data=AMOs[i].values, delta=1., time_bandwidth=4,
        number_of_tapers=5, statistics=True)

    ax.set_yscale('log')
    ax.set_xscale('log')

    ax.plot(1/freq, spec, color=f'C{i}', label=run.upper())
    ax.fill_between(1/freq, jackknife[:, 0], jackknife[:, 1], color=f'C{i}', alpha=0.1)

ax.legend(fontsize=14, loc=4, frameon=False)
ax.set_ylim((1E-6, 1E1))
ax.set_xlim((5, 2e3))
ax.set_xlabel('Period [yr]', fontsize=14)
ax.set_ylabel('Power Spectral Density', fontsize=14)

plt.tight_layout()


In [ ]:
np.shape(jackknife)

In [ ]:
(AMO_ctrl.time-31)/365

In [ ]:
((AMO_ctrl.shift(shifts={'time':1})-AMO_ctrl).time-31)/365

In [ ]:
type(AMO_ctrl)

In [ ]:
from pandas.tools.plotting import autocorrelation_plot

In [ ]:
autocorrelation_plot(AMO_ctrl)
autocorrelation_plot(AMO_ctrl)

In [ ]:
from statsmodels.tsa.ar_model import AR

In [ ]:
import xarray as xr


In [ ]:
ds = xr.open_dataset(f'{path_samoc}/SST/AMO_regr_had.nc')

In [ ]:
ds.slope.plot(vmax=.1)

In [ ]:
ds.pval.where(ds.pval<0.001).plot()
ds.pval.where(ds.pval>.999).plot()